# Initial Orbit Determination with `digest2`

#### This tutorial demonstrates how to use the `digest2` Python package for NEO orbit classification from short-arc astrometric tracklets.

`digest2` is a fast orbit classifier that assigns pseudo-probability scores (0–100) to astrometric tracklets for each of 14 orbit classes (NEO, Main Belt, Mars Crosser, etc.). It is the primary tool used by the Minor Planet Center to decide which tracklets are posted to the [NEO Confirmation Page (NEOCP)](https://minorplanetcenter.net/iau/NEO/toconfirm_tabular.html) for follow-up.

This can be useful if you want to:
- Quickly classify newly observed tracklets before submitting to the MPC
- Prioritize follow-up observations of potential NEO discoveries
- Understand how the MPC evaluates NEOCP candidates
- Incorporate orbit classification into automated survey pipelines

**References:**
- Keys et al. 2019, "The digest2 NEO Classification Code" ([PASP 131, 064501](https://arxiv.org/abs/1904.09188))
- Shober, Cloete, Veres 2023, "Improvement of digest2 NEO Classification Code" ([arXiv:2309.16407](https://arxiv.org/abs/2309.16407))
- Veres et al. 2025, "NEOCP Filters" ([arXiv:2505.11910](https://arxiv.org/abs/2505.11910))

# Install and Import

Install the `digest2` package from PyPI. This includes the compiled C scoring engine — no external C libraries are required.

In [ ]:
# pip install digest2 requests

In [ ]:
from digest2 import Digest2, Observation, classify, ClassificationResult, Scores
from digest2.observation import parse_mpc80_file, parse_ades_xml
import requests
import tempfile
import os
import atexit

# Setup: Download Sample Data

`digest2` requires an observatory codes file (`digest2.obscodes`) to look up parallax constants for each observatory. We download this from the MPC's [Observatory Codes API](https://docs.minorplanetcenter.net/mpc-ops-docs/apis/obscodes/) in the flat-file format.

We also create sample observation files in both MPC 80-column format and ADES XML format.

In [ ]:
# Create a temporary directory for our working files
tmpdir = tempfile.mkdtemp(prefix="digest2_tutorial_")
atexit.register(lambda: __import__('shutil').rmtree(tmpdir, ignore_errors=True))

# Download observatory codes from the MPC obscodes API (required by digest2)
# We request the flat-file format ("ObsCodes.html") which digest2 can parse directly.
# See: https://docs.minorplanetcenter.net/mpc-ops-docs/apis/obscodes/
response = requests.get(
    "https://data.minorplanetcenter.net/api/obscodes",
    json={"format": "ObsCodes.html"}
)
response.raise_for_status()

obscodes_path = os.path.join(tmpdir, "digest2.obscodes")
with open(obscodes_path, "w") as f:
    f.write(response.text)
print(f"Observatory codes downloaded to: {obscodes_path}")
print(f"  ({len(response.text.splitlines())} lines)")

In [ ]:
# Sample MPC 80-column observation file: 3 observations of 2016 SK99 from G96 (Mt. Lemmon)
sample_obs_content = """     K16S99K 1C2022 12 25.38496508 32 36.283+17 10 35.94         21.98GV     G96
     K16S99K 1C2022 12 25.39527308 32 35.635+17 10 37.27         21.72GV     G96
     K16S99K 1C2022 12 25.40040208 32 35.473+17 10 37.38         21.31GV     G96
"""

sample_obs_path = os.path.join(tmpdir, "sample.obs")
with open(sample_obs_path, "w") as f:
    f.write(sample_obs_content)

# Sample ADES XML observation file (same object, richer metadata)
sample_xml_content = """<?xml version="1.0" encoding="UTF-8"?>
<ades version="2017">
      <optical>
        <provID>2016 SK99</provID>
        <trkSub>C8QY322</trkSub>
        <mode>CCD</mode>
        <stn>G96</stn>
        <obsTime>2022-12-25T09:14:20.991Z</obsTime>
        <ra>128.151180</ra>
        <dec>17.176650</dec>
        <rmsRA>0.247</rmsRA>
        <rmsDec>0.301</rmsDec>
        <astCat>Gaia2</astCat>
        <mag>21.98</mag>
        <band>G</band>
      </optical>
      <optical>
        <provID>2016 SK99</provID>
        <trkSub>C8QY322</trkSub>
        <mode>CCD</mode>
        <stn>G96</stn>
        <obsTime>2022-12-25T09:29:11.606Z</obsTime>
        <ra>128.148480</ra>
        <dec>17.177020</dec>
        <rmsRA>0.431</rmsRA>
        <rmsDec>0.438</rmsDec>
        <astCat>Gaia2</astCat>
        <mag>21.72</mag>
        <band>G</band>
      </optical>
      <optical>
        <provID>2016 SK99</provID>
        <trkSub>C8QY322</trkSub>
        <mode>CCD</mode>
        <stn>G96</stn>
        <obsTime>2022-12-25T09:36:34.723Z</obsTime>
        <ra>128.147805</ra>
        <dec>17.177050</dec>
        <rmsRA>0.561</rmsRA>
        <rmsDec>0.573</rmsDec>
        <astCat>Gaia2</astCat>
        <mag>21.31</mag>
        <band>G</band>
      </optical>
</ades>
"""

sample_xml_path = os.path.join(tmpdir, "sample.xml")
with open(sample_xml_path, "w") as f:
    f.write(sample_xml_content)

print(f"Sample .obs file: {sample_obs_path}")
print(f"Sample .xml file: {sample_xml_path}")

# Basic Usage: Classifying a File

The simplest way to use `digest2` is to classify an observation file. The `Digest2` class loads the population model once and can then classify many tracklets efficiently.

We use it as a context manager (`with` statement) so that C resources are automatically released.

In [ ]:
with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
    results = d2.classify_file(sample_obs_path)

r = results[0]
print(f"Designation: {r.designation}")
print(f"Tracklet RMS: {r.rms:.2f} arcsec")
print()
print("NoID scores (pseudo-probability for each orbit class):")
for cls, val in r.noid.items():
    if round(val) > 0:
        print(f"  {cls:4s}: {val:5.1f}")

## Understanding the Results

Each result is a `ClassificationResult` dataclass with the following attributes:

| Attribute | Type | Description |
|-----------|------|-------------|
| `designation` | `str` | Object designation from the input file |
| `rms` | `float` | Great-circle RMS fit of the tracklet (arcseconds) |
| `noid` | `Scores` | **NoID scores**: pseudo-probabilities assuming the object is *unidentified* (0-100) |
| `raw` | `Scores` | **Raw scores**: pseudo-probabilities using total population (0-100) |
| `top_class` | `str` | The class with the highest NoID score |

The `Scores` object supports both attribute access (`result.noid.NEO`) and dict-style access (`result.noid["NEO"]`), as well as iteration via `.items()`.

The **NoID** scores (default) represent the probability that an unidentified tracklet belongs to each class, accounting for objects already discovered. These are the operationally relevant scores.

The 14 orbit classes are:

| Abbr | Class | Description |
|------|-------|-------------|
| Int | MPC Interest | q<1.3 OR e>0.5 OR i>=40 OR Q>10 |
| NEO | Near-Earth Object | q < 1.3 AU |
| N22 | Large NEO | NEO with H <= 22 |
| N18 | Very Large NEO | NEO with H <= 18 |
| MC | Mars Crosser | |
| Hun | Hungaria | |
| Pho | Phocaea | |
| MB1 | Inner Main Belt | |
| Pal | Pallas family | |
| Han | Hansa family | |
| MB2 | Middle Main Belt | |
| MB3 | Outer Main Belt | |
| Hil | Hilda group | |
| JTr | Jupiter Trojan | |
| JFC | Jupiter Family Comet | |

## Raw vs NoID Scores

Let's compare both score types to see the difference. Raw scores use the total population; NoID scores use only the *undiscovered* population.

In [ ]:
print(f"{'Class':>4s}  {'Raw':>6s}  {'NoID':>6s}")
print("-" * 22)
for cls in r.raw:
    raw_val = r.raw[cls]
    noid_val = r.noid[cls]
    if round(raw_val) > 0 or round(noid_val) > 0:
        print(f"{cls:>4s}  {raw_val:6.1f}  {noid_val:6.1f}")

# Programmatic Observations

Instead of reading from a file, you can construct `Observation` objects directly in Python. This is useful when integrating `digest2` into an automated pipeline.

Each observation requires:
- `mjd`: Modified Julian Date
- `ra`: Right Ascension in degrees
- `dec`: Declination in degrees
- `obscode`: MPC 3-character observatory code

Optional fields include `mag` (magnitude), `band` (photometric band), `rms_ra`, and `rms_dec` (astrometric uncertainties in arcseconds).

In [ ]:
obs = [
    Observation(mjd=59938.384965, ra=128.15118, dec=17.17665,
                mag=22.22, band="G", obscode="G96"),
    Observation(mjd=59938.395273, ra=128.14899, dec=17.17702,
                mag=21.96, band="G", obscode="G96"),
    Observation(mjd=59938.400402, ra=128.14780, dec=17.17717,
                mag=21.55, band="G", obscode="G96"),
]

with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
    result = d2.classify_tracklet(obs)

print(f"RMS: {result.rms:.2f} arcsec")
print()
for cls, val in result.noid.items():
    if round(val) > 0:
        print(f"  {cls:4s}: {val:5.1f}")

## One-Shot Classification

For quick, one-off classification, the `classify()` convenience function handles initialization and cleanup automatically. It is polymorphic — it accepts a filepath, a single tracklet, or a batch of tracklets.

In [ ]:
result = classify(obs, obscodes_path=obscodes_path, repeatable=True)

print(f"NEO score: {result.noid.NEO:.1f}")
print(f"MB1 score: {result.noid.MB1:.1f}")

# Input Formats: 80-Column vs ADES XML

The MPC uses two observation formats:
1. **MPC 80-column format** (`.obs`): Legacy fixed-width format
2. **ADES XML format** (`.xml`): Modern format with per-observation uncertainties

The `classify_file()` method auto-detects the format from the file extension. Let's classify the same object in both formats and compare.

In [ ]:
with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
    results_obs = d2.classify_file(sample_obs_path)
    results_xml = d2.classify_file(sample_xml_path)

r_obs = results_obs[0]
r_xml = results_xml[0]

print(f"{'':>4s}  {'80-col':>8s}  {'XML':>8s}")
print("-" * 26)
for cls in r_obs.noid:
    v_obs = r_obs.noid[cls]
    v_xml = r_xml.noid[cls]
    if round(v_obs) > 0 or round(v_xml) > 0:
        print(f"{cls:>4s}  {v_obs:8.1f}  {v_xml:8.1f}")

print(f"\n80-col designation: {r_obs.designation}")
print(f"XML designation:    {r_xml.designation}")

The scores are very similar between formats. Small differences can arise because:
- The ADES XML format includes per-observation astrometric uncertainties (`rmsRA`, `rmsDec`)
- The 80-column format relies on observatory-level default errors
- Designations may differ (80-column uses packed provisional designation, XML uses `trkSub`)

# Configuration: Observatory Error Models

The assumed astrometric uncertainty for each observatory significantly affects the scores. `digest2` supports three levels of error specification:

1. **Default error** (1.0 arcsec): Used when no config file is provided
2. **Per-site errors via `MPC.config`**: Observatory-specific errors calibrated from historical data
3. **Per-observation errors via ADES**: Individual uncertainties from the reduction pipeline

Let's compare scores with and without the `MPC.config` file. The bundled `MPC.config` specifies, for example, that observatory G96 (Mt. Lemmon) has a calibrated error of 0.29 arcsec — much better than the 1.0 arcsec default.

In [ ]:
# Create an empty config file (uses default 1.0 arcsec for all observatories)
empty_config_path = os.path.join(tmpdir, "empty.cfg")
with open(empty_config_path, "w") as f:
    f.write("# No per-site errors\n")

with Digest2(config_path=empty_config_path, obscodes_path=obscodes_path, repeatable=True) as d2:
    r_default = d2.classify_file(sample_obs_path)[0]

print("=== Default errors (1.0 arcsec for all sites) ===")
for cls, val in r_default.noid.items():
    if round(val) > 0:
        print(f"  {cls:4s}: {val:5.1f}")

In [ ]:
# Use MPC.config (auto-discovered, includes G96=0.29 arcsec)
with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
    r_mpc = d2.classify_file(sample_obs_path)[0]

print("=== MPC.config (G96 = 0.29 arcsec) ===")
for cls, val in r_mpc.noid.items():
    if round(val) > 0:
        print(f"  {cls:4s}: {val:5.1f}")

In [ ]:
header_default = 'Default (1.0")'
header_mpc = 'MPC.config (0.29")'
print("Comparison: Effect of observatory error model")
print(f"{'Class':>4s}  {header_default:>15s}  {header_mpc:>18s}")
print("-" * 42)
for cls in r_default.noid:
    v_def = r_default.noid[cls]
    v_mpc = r_mpc.noid[cls]
    if round(v_def) > 0 or round(v_mpc) > 0:
        print(f"{cls:>4s}  {v_def:15.1f}  {v_mpc:18.1f}")

With the default 1.0 arcsec error, the tracklet scores as predominantly main belt (MB1 ~ 85). With the calibrated G96 error of 0.29 arcsec, the position constraints are tighter, and the NEO score increases significantly. This demonstrates why per-site error calibration matters for accurate classification.

At the MPC, a NEO score of 65 or above triggers posting to the NEOCP.

## Custom Configuration Files

You can create a custom config file to set observatory errors for your own site. The format is simple: one `obserrXXX=Y.YY` line per observatory, where `XXX` is the MPC observatory code and `Y.YY` is the error in arcseconds.

In [ ]:
# Create a custom config with a very tight error for G96
tight_config_path = os.path.join(tmpdir, "tight.cfg")
with open(tight_config_path, "w") as f:
    f.write("obserrG96=0.10\n")  # Very precise astrometry

# Create a custom config with a very loose error for G96
loose_config_path = os.path.join(tmpdir, "loose.cfg")
with open(loose_config_path, "w") as f:
    f.write("obserrG96=2.0\n")  # Poor astrometry

with Digest2(config_path=tight_config_path, obscodes_path=obscodes_path, repeatable=True) as d2:
    r_tight = d2.classify_file(sample_obs_path)[0]

with Digest2(config_path=loose_config_path, obscodes_path=obscodes_path, repeatable=True) as d2:
    r_loose = d2.classify_file(sample_obs_path)[0]

h_tight = 'Tight (0.1")'
h_default = 'Default (1.0")'
h_loose = 'Loose (2.0")'
print(f"{'Class':>4s}  {h_tight:>13s}  {h_default:>15s}  {h_loose:>13s}")
print("-" * 52)
for cls in r_tight.noid:
    v_t = r_tight.noid[cls]
    v_d = r_default.noid[cls]
    v_l = r_loose.noid[cls]
    if round(v_t) > 0 or round(v_d) > 0 or round(v_l) > 0:
        print(f"{cls:>4s}  {v_t:13.1f}  {v_d:15.1f}  {v_l:13.1f}")

Tighter uncertainties constrain the range of possible orbits, which can sharpen the classification (increasing NEO probability for objects on NEO-like trajectories). Looser uncertainties allow more orbit solutions, generally pushing scores toward the most common population (Main Belt).

# Per-Observation Uncertainties

When using ADES format or constructing observations programmatically, you can provide per-observation astrometric uncertainties (`rms_ra`, `rms_dec`). This is more precise than using a single error value for the entire observatory.

In [ ]:
# Observations WITHOUT per-observation uncertainties
obs_no_rms = [
    Observation(mjd=59938.384965, ra=128.15118, dec=17.17665,
                mag=22.22, band="G", obscode="G96"),
    Observation(mjd=59938.395273, ra=128.14899, dec=17.17702,
                mag=21.96, band="G", obscode="G96"),
    Observation(mjd=59938.400402, ra=128.14780, dec=17.17717,
                mag=21.55, band="G", obscode="G96"),
]

# Same observations WITH per-observation uncertainties (from ADES)
obs_with_rms = [
    Observation(mjd=59938.384965, ra=128.15118, dec=17.17665,
                mag=22.22, band="G", obscode="G96",
                rms_ra=0.247, rms_dec=0.301),
    Observation(mjd=59938.395273, ra=128.14899, dec=17.17702,
                mag=21.96, band="G", obscode="G96",
                rms_ra=0.431, rms_dec=0.438),
    Observation(mjd=59938.400402, ra=128.14780, dec=17.17717,
                mag=21.55, band="G", obscode="G96",
                rms_ra=0.561, rms_dec=0.573),
]

with Digest2(config_path=empty_config_path, obscodes_path=obscodes_path, repeatable=True) as d2:
    r_no_rms = d2.classify_tracklet(obs_no_rms)
    r_with_rms = d2.classify_tracklet(obs_with_rms, is_ades=True)

print(f"{'Class':>4s}  {'No per-obs RMS':>15s}  {'With per-obs RMS':>17s}")
print("-" * 40)
for cls in r_no_rms.noid:
    v1 = r_no_rms.noid[cls]
    v2 = r_with_rms.noid[cls]
    if round(v1) > 0 or round(v2) > 0:
        print(f"{cls:>4s}  {v1:15.1f}  {v2:17.1f}")

When per-observation uncertainties are provided with `is_ades=True`, `digest2` uses those values instead of the observatory-level defaults. Set `is_ades=True` to tell `digest2` to use the per-observation RMS values rather than the configured site error.

# Class Filtering

If you only care about specific orbit classes, use the `classes` parameter to compute only those scores. This can be useful for focused analyses.

In [ ]:
with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
    result = d2.classify_tracklet(obs, classes=["NEO", "MC", "MB1"])

print("Filtered classification (NEO, MC, MB1 only):")
print(f"  NEO: {result.noid.NEO:.1f}")
print(f"  MC:  {result.noid.MC:.1f}")
print(f"  MB1: {result.noid.MB1:.1f}")

Note that when filtering, scores are renormalized over the selected classes only, so the values will differ from the unfiltered case.

# Batch Classification

Use `classify_batch()` to classify multiple tracklets efficiently with a single model load.

In [ ]:
# Two different tracklets
tracklet_1 = [
    Observation(mjd=59938.384965, ra=128.15118, dec=17.17665,
                mag=22.22, band="G", obscode="G96"),
    Observation(mjd=59938.395273, ra=128.14899, dec=17.17702,
                mag=21.96, band="G", obscode="G96"),
    Observation(mjd=59938.400402, ra=128.14780, dec=17.17717,
                mag=21.55, band="G", obscode="G96"),
]

tracklet_2 = [
    Observation(mjd=59938.384965, ra=130.0, dec=20.0,
                mag=20.0, obscode="G96"),
    Observation(mjd=59938.395273, ra=130.01, dec=20.01,
                mag=20.0, obscode="G96"),
]

with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
    batch_results = d2.classify_batch([tracklet_1, tracklet_2])

for i, r in enumerate(batch_results):
    if r is not None:
        top_class = r.top_class
        top_score = r.noid[top_class]
        neo_score = r.noid.NEO
        print(f"Tracklet {i+1}: top class = {top_class} ({top_score:.1f}), NEO = {neo_score:.1f}")

# Parsing Observation Files

The parsing functions can be used independently, which is useful for inspecting observations before classification.

In [ ]:
# Parse MPC 80-column file
tracklets_80 = parse_mpc80_file(sample_obs_path)

for desig, obs_list in tracklets_80.items():
    print(f"Designation: '{desig.strip()}'  ({len(obs_list)} observations)")
    for o in obs_list:
        print(f"  MJD={o.mjd:.6f}  RA={o.ra:.5f}  Dec={o.dec:.5f}  "
              f"Mag={o.mag:.2f}  Site={o.obscode}")

In [ ]:
# Parse ADES XML file
tracklets_xml = parse_ades_xml(sample_xml_path)

for desig, obs_list in tracklets_xml.items():
    print(f"Designation: '{desig}'  ({len(obs_list)} observations)")
    for o in obs_list:
        print(f"  MJD={o.mjd:.6f}  RA={o.ra:.6f}  Dec={o.dec:.6f}  "
              f"Mag={o.mag:.2f}  Site={o.obscode}  "
              f"rmsRA={o.rms_ra:.3f}  rmsDec={o.rms_dec:.3f}")

Note that ADES XML provides per-observation astrometric uncertainties (`rmsRA`, `rmsDec`), which the 80-column format does not.

# Error Handling

`digest2` raises standard Python exceptions for common error cases.

In [ ]:
# Error: too few observations (need at least 2)
try:
    with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
        d2.classify_tracklet([Observation(mjd=59938.0, ra=128.0, dec=17.0, obscode="G96")])
except (RuntimeError, ValueError) as e:
    print(f"{type(e).__name__}: {e}")

# Error: invalid orbit class
try:
    with Digest2(obscodes_path=obscodes_path, repeatable=True) as d2:
        d2.classify_tracklet(obs, classes=["INVALID"])
except ValueError as e:
    print(f"ValueError: {e}")

# Error: using a closed instance
try:
    d2 = Digest2(obscodes_path=obscodes_path)
    d2.close()
    d2.classify_tracklet(obs)
except RuntimeError as e:
    print(f"RuntimeError: {e}")

# Summary

The `digest2` Python package provides fast orbit classification for short-arc astrometric tracklets. Key points:

- **Install**: `pip install digest2`
- **Basic usage**: `Digest2` class as a context manager, or `classify()` one-shot function
- **Input formats**: MPC 80-column (`.obs`) and ADES XML (`.xml`) files, or programmatic `Observation` objects
- **Configuration**: Per-site observatory errors via `MPC.config` (bundled) or custom config files
- **Uncertainties matter**: Smaller assumed errors produce tighter orbital constraints and sharper classification
- **NEO threshold**: Objects with NoID NEO score ≥ 65 are posted to the MPC's NEO Confirmation Page

For more information:
- [digest2 documentation on GitHub](https://github.com/Smithsonian/mpc-public/tree/main/digest2)
- [NEO Confirmation Page](https://minorplanetcenter.net/iau/NEO/toconfirm_tabular.html)
- [Keys et al. 2019](https://arxiv.org/abs/1904.09188) — Algorithm description

For questions or feedback, contact the MPC via the [Jira Helpdesk](https://mpc-service.atlassian.net/servicedesk/customer/portal/13/create/148).